In [ ]:
from init_notebook import *
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer,
    GenerationConfig
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    trust_remote_code=True,
    load_in_4bit=True,
)

In [ ]:
model

In [ ]:
sum(math.prod(p.shape) for p in model.parameters())

In [ ]:
f"params: {num_module_parameters(model):,}"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

In [ ]:
len(tokenizer.get_vocab())

In [ ]:
tokenizer.encode("Text1")

In [ ]:
def generate(prompt: str):
    next_id = tokenizer.encode(prompt, return_tensors="pt")
    past_key_values = None
    while True:
        next_logits, past_key_values = model(
            next_id, use_cache=True, past_key_values=past_key_values,
        ).to_tuple()
        
        next_logits = next_logits[:, -1:]

        sorted = next_logits.argsort(dim=-1)
        next_id = next_logits.argmax(dim=-1)
        #next_id = random.choice(sorted[..., :2])
        #print(next_id)

        if next_id == tokenizer.eos_token_id:
            break

        yield tokenizer.decode(next_id.item())

for token in generate(
        #"Question: How to change the temperature of a language model sampler?\n",
        "Exercise: Write a representative Python program about Karl Marx's \"Capital\"\n",
):
    print(token, end="")

In [ ]:
class Streamer:
    def put(self, token_ids):
        for token_id in token_ids.flatten(0):
            print(tokenizer.decode(token_id), end="")
    def end(self):
        print("END")

def generate2(prompt: str):
    token_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    model.generate(
        inputs=token_ids,
        streamer=Streamer(),
        generation_config=GenerationConfig(
            max_new_tokens=100,
            do_sample=True,
            temperature=1.,
        ),
    )
    
generate2(
    "Question: How to change the temperature of a language model sampler?\n",
    
)
#    print(token, end="")

In [ ]:
GenerationConfig?

In [ ]:
model.generate?